In [ ]:
# coding=utf-8

import glob, os
import sys, re
import xml.etree.ElementTree as ET

mapa = sys.argv[1]
stNepravilnoOdprtih = 0
raw_text = ""
file_path = ""
i = 0
file_counter = 0
file_in_counter = 0
najdaljsiTrStavek = 0
katerJeNajdaljsi = ""
skupnaDolzinaStavkov = 0
stStavkov = 0
fileNameFolders = glob.glob("/content/drive/My Drive/ucenje_BERT/gigafida/tei/GF*")

skip_stevec = 0

koliko_besed = 0

for fileNameFolder in fileNameFolders:
  if skip_stevec < 5:
    skip_stevec += 1
    continue

  fileNames = glob.glob(fileNameFolder + "/*.xml")
  for fileName in fileNames:

    if file_in_counter > 15:
      break
    file_in_counter += 1

    tree = ET.parse(fileName)
    root = tree.getroot()

    text = ""

    for root_text in root[1][1]:
      for child in root_text:
        for child_with_text in child:
          if child_with_text.get('name') != 'neardup' and child_with_text.get('name') != 'std_tech_n' and child_with_text.get('name') != 'std_ling_n':
            #print(child_with_text.text)
            text += child_with_text.text
        text += '\n'

    if (i % 10 == 0):
        print(i)
    i += 1

    if not ((text.count('\n') / len(text) > 0.05) or (sum(c.isdigit() for c in text) / len(text) > 0.03) or (text.count(':') / len(text) > 0.01) or (text.count('(') / len(text) > 0.01)):


        #text = text.lower()
        text = re.sub('[!@#$:;\'\"“”•«»%&ø×¹]', '', text)
        text = re.sub('[öó]', 'o', text)
        text = re.sub('[áä]', 'a', text)
        text = re.sub('[0123456789]', '1', text)
        text = re.sub('ë', 'e', text)
        text = re.sub('í', 'i', text)
        text = re.sub('ü', 'u', text)
        text = re.sub('ć', 'c', text)


        #odstranim odvecne presledke pred locili
        text0 = ""
        c1 = ''
        c2 = ''
        for c in text:
            c1 = c
            if ((c1 == '.' or c1 == ',' or c1 == '?' or c1 == '!') and c2 != ' ') or (c1 != '.' and c1 != ',' and c1 != '?' and c1 != '!'):
                text0 += c2
            c2 = c


        #odstranim veckratne presledke iz teksta
        text1 = ""
        c1 = ''
        c2 = ''
        for c in text0:
            c1 = c
            if not (c1 == ' ' and c2 == ' '):
                text1 += c
            c2 = c


        #odstranim stevilke z vec enicami iz teksta -> vsa stevila so enice
        text2 = ""
        c1 = ''
        c2 = ''
        for c in text1:
            c1 = c
            if not (c1 == '1' and c2 == '1'):
                text2 += c
            c2 = c


        #dodam presledke za stevilke
        text25 = ""
        c2 = ''
        for c in text2:
            if (c2 == '1' and (c != ' ' and c != ',' and c != '.')):
                text25 += c2
                text25 += ' '
            else:
                text25 += c2
            c2 = c

        #dodam presledke po piki, ce manjkajo
        text26 = ""
        c2 = ''
        for c in text25:
            if ((c2 == ',' or c2 == '.') and (c != ' ' and c != '\n' and c != '1')):
                text26 += c2
                text26 += ' '
            else:
                text26 += c2
            c2 = c

        text2 = text26
        #izlocim stavke z *, +, -
        text3 = ""
        vmesniText = ""
        bilPlusKratMinusZnak = False
        for c in text2:
            vmesniText += c
            if c == '+' or c == '-' or c == '*':
                bilPlusKratMinusZnak = True
            elif c == '.' or c == '?' or c == '!':
                if not bilPlusKratMinusZnak:
                    text3 += vmesniText
                bilPlusKratMinusZnak = False
                vmesniText = ""

        text4 = ""
        splittedText = text3.split('\n');
        for stavek in splittedText:
          if len(stavek) > 1:
            if "." in stavek:
              text4 += stavek + "\n"

        text6 = text4

        text7 = ""
        c1 = ''
        c2 = ''
        for c in text6:
            c1 = c
            if ((c2 == ',' or c2 == '.' or c2 == '!' or c2 == '?') and c1 == '1'):
                text7 += ' ' + c
            else:
                text7 += c
            c2 = c

        #odstranim se odvecne presledke
        text5 = ""
        c1 = ''
        c2 = ''
        for c in text7:
            c1 = c
            if not (c1 == ' ' and c2 == ' '):
                text5 += c
            c2 = c
        #text5 = text4

        #print("text5 je" + text5)

        if len(text5) > 10:
            raw_text += text5
            stNepravilnoOdprtih = 0

  #print(raw_text)
  file_path = "/content/drive/My Drive/magistrska/poskusno_generiranje_texta/poskusno_generiranje_texta" + str(file_counter) + ".txt"
  file = open(file_path,"w")
  file.write(raw_text)
  file.close()
  raw_text = ""
  if file_counter > 0:
    break
  file_counter += 1





#skupnaDolzinaStavkov /= stStavkov

print("povprecnaDolzinaStavkov je", skupnaDolzinaStavkov)
print("najdalji tr stavek je", najdaljsiTrStavek)
print(katerJeNajdaljsi)

In [ ]:
# coding: utf-8

from __future__ import division, print_function
from nltk.tokenize import word_tokenize

import nltk
import os
from io import open
import re
import sys

nltk.download('punkt')

NUM = '<NUM>'

EOS_PUNCTS = {".": ".PIKA", "?": ".PIKA", "!": ".PIKA"}
INS_PUNCTS = {",": ",VEJICA", ";": ".PIKA", ":": " ", "-": " "}

forbidden_symbols = re.compile(r"[\[\]\(\)\/\\\>\<\=\+\_\*]")
numbers = re.compile(r"\d")
multiple_punct = re.compile(r'([\.\?\!\,\:\;\-])(?:[\.\?\!\,\:\;\-]){1,}')

is_number = lambda x: len(numbers.sub("", x)) / len(x) < 0.6

def untokenize(line):
    return line.replace(" '", "'").replace(" n't", "n't").replace("can not", "cannot")

def skip(line):

    if line.strip() == '':
        return True

    last_symbol = line[-1]
    if not last_symbol in EOS_PUNCTS:
        return True

    if forbidden_symbols.search(line) is not None:
        return True

    return False

def process_line(line):

    # tokenized line looks like words in array, for example: ["I", "am", ....]
    tokens = word_tokenize(line)
    output_tokens = []

    for token in tokens:

        if token in INS_PUNCTS:
            output_tokens.append(INS_PUNCTS[token])
        elif token in EOS_PUNCTS:
            output_tokens.append(EOS_PUNCTS[token])
        elif is_number(token):
            output_tokens.append(NUM)
        else:
            # we get bach normal sentance with tokens
            output_tokens.append(token)

    return untokenize(" ".join(output_tokens) + " ")

skipped = 0
allLines = 0

import glob, os
fileNames = glob.glob("/content/drive/My Drive/ucenje_BERT/proba_pred_zagovorom/*")

file_counter = 0
for filename in fileNames:
  noviFilename = "/content/drive/My Drive/ucenje_BERT/proba_pred_zagovorom/vmesniPreparedText" + str(file_counter) + ".txt"
  file_counter += 1
  with open(noviFilename, 'w', encoding='utf-8') as out_txt:
      with open(filename, 'r', encoding='utf-8') as text:

        print("vsaj tuki sm!!")
        for line in text:
            #print("line je --> ", line)
            line = line.replace("\"", "").strip()
            line = multiple_punct.sub(r"\g<1>", line)

            allLines += 1
            if skip(line):
                skipped += 1
                continue

            line = process_line(line)

            out_txt.write(line + '\n')

print("Skipped %d lines" % skipped)
print("All %d lines" % allLines)

In [ ]:
pustiVejice = False

import glob, os
#fileNames = glob.glob("/content/drive/My Drive/ucenje_BERT/nova_ucna_mnozica_bert/vmesniPreparedText*")
fileNames = glob.glob("/content/drive/My Drive/ucenje_BERT/testna_mnozica_bert/testna_mnozica_halozan/vejica_stavki.txt")

file_counter = 0
for filename in fileNames:

  output_file = "/content/drive/My Drive/ucenje_BERT/testna_mnozica_bert/testna_mnozica_halozan/vmesni2PreparedText" + str(file_counter) + ".txt"
  file_counter += 1
  pustiVejice = True

  with open(filename, 'r', encoding='utf-8') as input_text:

    lines = input_text.read().split('\n')
    all_lines = ""

    stevec = 0
    vseh = len(lines)

    for line in lines:

        print('it is')

        stevec += 1
        print('progress: ', stevec / vseh, '%')

        if pustiVejice:
            line = line.replace(',VEJICA .PIKA', '.').replace(',VEJICA', ',VEJICA').replace('.PIKA', '.').replace('?QUESTIONMARK', '?').replace('<NUM>', '400')
        else:
            line = line.replace(' ,VEJICA', '').replace('.PIKA', '.').replace('?VPRASAJ', '?')


        line = "[CLS] " + line + "[SEP]\n"
        all_lines += line

f = open(output_file, "w")
f.write("%s" % all_lines)
f.close()

In [ ]:
import csv

def copy_line_add_mask(line_to_copy, index_add_mask, labels):
    copied_line = ""
    counter = 0
    masked_comma = False
    mask_next_one = False
    added_token_previousely = False
    beginning = True
    ending = False
    added_label_previously = False


    for token in line_to_copy:
        if index_add_mask != -1:
            if counter != 0 and token != ',VEJICA':
                copied_line += " "

            if token != ',VEJICA':
                copied_line += token
            elif counter == index_add_mask:
                masked_comma = True

            if counter == index_add_mask:
                copied_line += ' [MASK]'
            counter += 1
        else:
            if token == '.':
                ending = True

            if not ending and not mask_next_one and added_token_previousely:
                copied_line += ' [MASK] '
            elif ending:
                copied_line += ' '

            if added_label_previously and not ending and counter > 2 and token != ',VEJICA':
                # copied_line += " "
                labels += " "
                added_label_previously = False

            if token != ',VEJICA':
                copied_line += token
                if counter != 0:
                    added_token_previousely = True
                else:
                    copied_line += ' '
                if not beginning and not ending:
                    if not mask_next_one:
                        labels += "0"
                        added_label_previously = True
                    else:
                        mask_next_one = False
                        labels += "1"
                        added_label_previously = True
                elif counter != 0:
                    beginning = False
            else:
                mask_next_one = True
                added_token_previousely = True
            counter += 1

    if index_add_mask != -1:
        if masked_comma:
            labels += "1\n"#"VEJICA\n"
        else:
            labels += "0\n"#"PRESLEDEK\n
    else:
        labels += "\n"

    return (copied_line, labels)


import glob, os
fileNames = glob.glob("/content/drive/My Drive/ucenje_BERT/testna_mnozica_bert/testna_mnozica_halozan/vmesni2PreparedText*")

file_counter = 0
#print(fileNames)

for filename in fileNames:

  mask_all_in_one_sentence = True
  add_training_labels = False

  output_file = "/content/drive/My Drive/ucenje_BERT/testna_mnozica_bert/testna_mnozica_halozan/maskedPreparedText" + str(file_counter) + ".txt"
  file_counter += 1

  add_training_labels = True

  skipped_lines = 0
  matrix_of_masked_lines = ""
  matrix_of_labels = ""

  pripravljen_text_bert_stavki = open(filename, 'r', encoding='utf-8').read()

  lines = pripravljen_text_bert_stavki.split('\n')
  stevec = 0
  vseh = len(lines)
  print(len(lines))

  for line in lines:
      #print(stevec)
      stevec += 1
      if stevec % 10000 == 0:
        print('progress: ', stevec / vseh, '%')

      #if (stevec > 10):
      #    break

      tokenized_line = line.split(' ')


      if tokenized_line[0] != '[CLS]' or tokenized_line[len(tokenized_line) - 1] != '[SEP]':
          skipped_lines += 1
          continue

      if not mask_all_in_one_sentence:
          for index in range(1, len(tokenized_line) - 3):
              new_masked_line, matrix_of_labels = copy_line_add_mask(tokenized_line, index, matrix_of_labels)
              new_masked_line = new_masked_line + "\n"
              matrix_of_masked_lines += new_masked_line
      else:
          new_masked_line, matrix_of_labels = copy_line_add_mask(tokenized_line, -1, matrix_of_labels)
          new_masked_line = new_masked_line + "\n"
          matrix_of_masked_lines += new_masked_line
      if not add_training_labels:
          matrix_of_masked_lines += "\nT\n"


  if add_training_labels:
      f = open(output_file, "w")
      print("v add new labels")
      #writer = csv.writer(f)

      sentance_rows = matrix_of_masked_lines.split('\n')
      label_rows = matrix_of_labels.split('\n')

      assert len(sentance_rows) == len(label_rows)

      for i in range (0, len(sentance_rows)):
          #print(sentance_rows[i] + "\t" + label_rows[i])
          if len(sentance_rows[i].split(' ')) > 1 and len(sentance_rows[i].split(' ')) <= 64:
              #print(sentance_rows[i] + "\t" + label_rows[i])
              f.write(sentance_rows[i] + "\t" + label_rows[i] + "\n")
          #writer.writerow(sentance_rows[i] + "\t" + label_rows[i])
      f.close()
  else:
      f = open(output_file, "w")
      f.write("%s" % matrix_of_masked_lines)
      f.close()
      print("Lines skipped", skipped_lines)


104186
progress:  0.0959821857063329 %
progress:  0.1919643714126658 %
progress:  0.2879465571189987 %
progress:  0.3839287428253316 %
progress:  0.4799109285316645 %
progress:  0.5758931142379974 %
progress:  0.6718752999443304 %
progress:  0.7678574856506633 %
progress:  0.8638396713569961 %
progress:  0.959821857063329 %
v add new labels


In [ ]:
!pip install Cython
!pip install torch
import torch

# If there's a GPU available...
if torch.cuda.is_available():

    # Tell PyTorch to use the GPU.
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
import pandas as pd

df = []

# Load the dataset into a pandas dataframe.
li = pd.read_csv('/content/drive/My Drive/ucenje_BERT/nova_ucna_mnozica_bert/maskedPreparedText0.txt', delimiter='\t', header=None, names=['sentence', 'label'])
df.append(li)
li = pd.read_csv('/content/drive/My Drive/ucenje_BERT/nova_ucna_mnozica_bert/maskedPreparedText1.txt', delimiter='\t', header=None, names=['sentence', 'label'])
df.append(li)
li = pd.read_csv('/content/drive/My Drive/ucenje_BERT/nova_ucna_mnozica_bert/maskedPreparedText2.txt', delimiter='\t', header=None, names=['sentence', 'label'])
df.append(li)
li = pd.read_csv('/content/drive/My Drive/ucenje_BERT/nova_ucna_mnozica_bert/maskedPreparedText3.txt', delimiter='\t', header=None, names=['sentence', 'label'])
df.append(li)
li = pd.read_csv('/content/drive/My Drive/ucenje_BERT/nova_ucna_mnozica_bert/maskedPreparedText4.txt', delimiter='\t', header=None, names=['sentence', 'label'])
df.append(li)

df = pd.concat(df, axis=0, ignore_index=True)
#
df.drop(df.tail(2).index,inplace=True)
# Report the number of sentences.
print('Number of training sentences: {:,}\n'.format(df.shape[0]))

# Display 10 random rows from the data.
df.sample(100)

In [ ]:
# Get the lists of sentences and their labels.
sentences = df.sentence.values
labels = df.label.values
print(labels)

In [ ]:
!pip install transformers
from transformers import BertTokenizer

#SLO
tokenizer = BertTokenizer.from_pretrained('drive/My Drive/ucenje_BERT/pretrained_slo_bert/', do_lower_case=True)

#MULTILINGUAL
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True)


In [ ]:
sent = "[CLS] Ko [MASK] se [MASK] je [MASK] obrnila [MASK] je [MASK] bila [MASK] pretresena . [SEP]"

# Print the original sentence.
print(' Original: ', sent)

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sent))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent)))

In [ ]:
max_len = 0

# For every sentence...
stevec = 0

for sent in sentences:
  stevec +=1
  #if stevec > 1000:
    #break
  # Tokenize the text and add `[CLS]` and `[SEP]` tokens.

  try:
    input_ids110 = tokenizer.encode(sent, add_special_tokens=False)
  except:
    print(stevec)
    print(sent)
    print(input_ids110)

  # Update the maximum sentence length.
  max_len = max(max_len, len(input_ids110))

print(stevec)
print('Max sentence length: ', max_len)

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
import torch
import numpy as np

input_ids = []
attention_masks = []
label_list = []
stevec = 0
stevec_izpuscenih = 0
print(len(sentences))
# For every sentence...
for sent in sentences:
  #if stevec > 100:
  #  break
  stevec += 1

  if (stevec % 10000 == 0):
    print("progress: ", stevec / len(sentences))

  #sent = "[CLS] Ko [MASK] se [MASK] je [MASK] obrnila [MASK] je [MASK] bila [MASK] pretresena . [SEP]"
  # `encode_plus` will:
  #   (1) Tokenize the sentence.
  #   (2) Prepend the `[CLS]` token to the start.
  #   (3) Append the `[SEP]` token to the end.
  #   (4) Map tokens to their IDs.
  #   (5) Pad or truncate the sentence to `max_length`
  #   (6) Create attention masks for [PAD] tokens.
  encoded_dict = tokenizer.encode_plus(
                      sent,                       # Sentence to encode.
                      add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                      max_length = 128,           # Pad & truncate all sentences.
                      pad_to_max_length = True,
                      truncation = True,
                      return_attention_mask = True,   # Construct attn. masks.
                      return_tensors = 'pt',      # Return pytorch tensors.
                  )



  try:

    labels_in = []
    cur_labels = labels[stevec - 1].split(' ')

    maskedindexesIndexPointer = 0

    for token in encoded_dict['input_ids'][0].numpy():
      if token == 105:
        labels_in.append(int((cur_labels[maskedindexesIndexPointer])))
        maskedindexesIndexPointer += 1
      else:
        labels_in.append(-100)

    #print(encoded_dict['input_ids'][0].numpy())

    #print(encoded_dict['attention_mask'][0].numpy())

    #print(labels_in)

    assert len(encoded_dict['input_ids'][0].numpy()) == len(labels_in)

    #print(labels_in)

    if maskedindexesIndexPointer != len(cur_labels):
      #print(sent)
      #print(maskedindexesIndexPointer)
      #print(encoded_dict['input_ids'])
      #print(cur_labels)
      stevec_izpuscenih += 0

    #print(maskedindexesIndexPointer)
    #print(len(cur_labels))

    assert maskedindexesIndexPointer == len(cur_labels)
    #print(encoded_dict['input_ids'])

    # Add the encoded sentence to the list.
    input_ids.append(encoded_dict['input_ids'])

    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

    label_list.append(torch.tensor([labels_in]))

  except:
    #if not type(labels[stevec - 1]) is float:
      #print(labels[stevec - 1].split(' '))
    #print(labels[stevec - 1])
    #print(encoded_dict['input_ids'])
    #print(sent)
    stevec_izpuscenih += 1

  #break


# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
#labels = torch.tensor(labels)
labels_torch = torch.cat(label_list, dim = 0)

# Print sentence 0, now as a list of IDs.
print('Original: ', tokenizer.tokenize(sentences[0]))
print('Token IDs:', input_ids[0])
print('Labels:', labels_torch[0])
print('Original: ', tokenizer.tokenize(sentences[4]))
print('Token IDs:', input_ids[4])
print('Labels:', labels_torch[4])
print('Original: ', tokenizer.tokenize(sentences[400]))
print('Token IDs:', input_ids[400])
print('Labels:', labels_torch[400])

print("st izpuscenih")
print(stevec_izpuscenih)

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels_torch)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.900 * len(dataset))
val_and_test_size = len(dataset) - train_size

val_size = int(0.5 * val_and_test_size)
test_size = int(0.5 * val_and_test_size)

print(train_size)
print(val_size)
print(test_size)

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

2389136
49774
49774
2,389,136 training samples
49,774 validation samples


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# size of 16 or 32.
batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )


validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


test_dataloader = DataLoader(
            test_dataset, # The test samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import BertForSequenceClassification, BertForMaskedLM, AdamW, BertConfig

#slo
model = BertForMaskedLM.from_pretrained(
    'drive/My Drive/ucenje_BERT/pretrained_slo_bert/'
)

#ne slo
#model = BertForMaskedLM.from_pretrained(
#    'bert-base-multilingual-cased'
#)

"""
model = BertForMaskedLM.from_pretrained(
    'drive/My Drive/ucenje_BERT/pretrained_slo_bert/',
    num_labels = 3,
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)
"""

# Tell pytorch to run this model on the GPU.
model.cuda()

In [ ]:
from transformers import BertForSequenceClassification, BertForMaskedLM, AdamW, BertConfig

In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
from transformers import BertForSequenceClassification, BertForMaskedLM, AdamW, BertConfig

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4.
epochs = 4
#epochs = 2

# Total number of training steps is [number of batches] x [number of epochs].
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

print(scheduler)
print(scheduler.get_last_lr())
#print(scheduler.step())
#print(scheduler.base_lrs)

[2e-05]


In [ ]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
import time
import datetime

def format_time(elapsed):
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))

    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import random
import numpy as np
import os, time

seed_val = 42

output_dir = 'drive/My Drive/ucenje_BERT/bert_multilingual/'

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_stevec = 0
stevci = []
learningRates = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):

    print("")
    print('Epoch {:} / {:}'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):


        #print(1)
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)

            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        loss, logits = model(b_input_ids,
                             token_type_ids=None,
                             attention_mask=b_input_mask,
                             labels=b_labels)

        total_train_loss += loss.item()

        loss.backward()


        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


        optimizer.step()

        # Update the learning rate.
        scheduler.step()
        total_stevec += 1

        if total_stevec % 1000 == 0:
          learningRates.append(scheduler.get_last_lr())
          stevci.append(total_stevec)

          print(scheduler.get_last_lr())

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

    print("")
    print("Running Validation...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    stevec = 0

    num_right_predictions = 0
    num_all_predictions = 0
    num_right_comma_predictions = 0
    num_all_commas = 0

    for batch in validation_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        batch_numpy = tuple(t.numpy() for t in batch)
        b_input_numpy_ids, b_input_numpy_mask, b_numpy_labels = batch_numpy

        with torch.no_grad():

            logits = model(b_input_ids,
                                   token_type_ids=None,
                                   attention_mask=b_input_mask)

            logits2 = logits[0]
            for batch_index in range(0, len(b_input_numpy_ids)):
            #print(batch_index)
              for i in range(0, len(b_input_numpy_ids[batch_index])):
                #print(torch.argmax(logits2[0, i]).item())
                if b_input_numpy_ids[batch_index][i] == 105:
                  if b_numpy_labels[batch_index][i] == torch.argmax(logits2[batch_index, i]).item():
                    num_right_predictions += 1
                  num_all_predictions += 1

                  if b_numpy_labels[batch_index][i] == 1:
                    if torch.argmax(logits2[batch_index, i]).item() == 1:
                      num_right_comma_predictions += 1
                    num_all_commas += 1

            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits[0].detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            pred = np.argmax(logits, axis=2).flatten()
            origin = label_ids.flatten()

            total_eval_accuracy += flat_accuracy(logits, label_ids)


    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)

    validation_time = format_time(time.time() - t0)

    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

    print('    DONE.')
    print('Accuracy: ' + str((num_right_predictions / num_all_predictions) * 100) + '%')
    print('Accuracy, comma: ' + str((num_right_comma_predictions / num_all_commas) * 100) + '%')
    print(num_right_predictions)
    print(num_all_predictions)
    print(num_right_comma_predictions)
    print(num_all_commas)

print("")
print("Training complete!")

print(stevci)
print(learningRates)

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [ ]:
import os

output_dir = 'drive/My Drive/ucenje_BERT/bert_multilingual_cased/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

In [ ]:
import re

def tokens_to_real_stavek(tokens, kje):
  stavek = ""
  stevec = 1
  for token in tokens:
    if token != '[MASK]' and token != '[SEP]' and token != '[CLS]':
      if stevec == kje:
        stavek += '!!!'
      if '##' not in token:
        stavek += " "
      stavek += re.sub('#', '', token)
      stevec += 1
  return stavek

In [ ]:
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables
predictions , true_labels = [], []

stevec = 0

num_right_predictions = 0
num_all_predictions = 0
num_right_comma_predictions = 0
num_all_commas = 0
should_be_comma = 0
should_be_space = 0

# Predict

print(len(test_dataloader))
for batch in test_dataloader:
  stevec += 1
  #print(stevec)
  if stevec < 500:
    continue
  if stevec % 10 == 0:
    print(stevec)
  # Add batch to GPU
  batch_numpy = tuple(t.numpy() for t in batch)
  b_input_numpy_ids, b_input_numpy_mask, b_numpy_labels = batch_numpy


  batch = tuple(t.to(device) for t in batch)

  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch

  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids,
                      attention_mask=b_input_mask)



  #print(outputs)
  logits = outputs[0]
  #print(logits[0])
  output_labels = []
  #batch_index = 9
  for batch_index in range(0, len(b_input_numpy_ids)):
    #print(batch_index)
    for i in range(0, len(b_input_numpy_ids[batch_index])):
      #print(torch.argmax(logits[0, i]).item())
      if b_input_numpy_ids[batch_index][i] == 103:
        if b_numpy_labels[batch_index][i] == torch.argmax(logits[batch_index, i]).item():
          num_right_predictions += 1
        elif b_numpy_labels[batch_index][i] == 0:
          should_be_space += 1
        else:
          should_be_comma += 1
        num_all_predictions += 1

        if b_numpy_labels[batch_index][i] == 1:
          if torch.argmax(logits[batch_index, i]).item() == 1:
            num_right_comma_predictions += 1
          num_all_commas += 1

  #break
  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()

  #print(np.argmax(logits, axis=1))
  #print(label_ids)

  # Store predictions and true labels
  predictions.append(np.argmax(logits, axis=1).flatten())
  true_labels.append(label_ids)
  #break
  if stevec > 1000:
    break

print('pravilne: ', num_right_predictions)
print('vse: ', num_all_predictions)
print('pravilne comma: ', num_right_comma_predictions)
print('vse comma: ', num_all_commas)
print('should be comma but model didnt place it: ', should_be_comma)
print('model predicted comma wrongly: ', should_be_space)

print('    DONE.')
print('Accuracy: ' + str((num_right_predictions / num_all_predictions) * 100) + '%')
print('Accuracy, comma: ' + str((num_right_comma_predictions / num_all_commas) * 100) + '%')

In [ ]:
print('Positive samples: %d of %d (%.2f%%)' % (df.label.sum(), len(df.label), (df.label.sum() / len(df.label) * 100.0)))

In [ ]:
!pip install transformers
from transformers import BertForSequenceClassification, BertForMaskedLM, AdamW, BertConfig, BertTokenizer

output_dir = 'drive/My Drive/ucenje_BERT/bert_multilingual_KONCEN/'

# Load a trained model and vocabulary that you have fine-tuned
model = BertForMaskedLM.from_pretrained(output_dir)
#tokenizer = BertTokenizer.from_pretrained(output_dir)

# Copy the model to the GPU.
model.to(device)

In [ ]:
!pip install transformers
from transformers.modeling_bert import BertForMaskedLM, BertModel

import sys, re

import time

import os

from transformers import BertTokenizer

!pip install torch
import torch

f = open("drive/My Drive/ucenje_BERT/testna_mnozica_bert/testna_mnozica_halozan/unzipped_testiraj_bert/vejica_stavki.txt", "r")
stavki = f.read()

tokenizer = BertTokenizer.from_pretrained('drive/My Drive/ucenje_BERT/testna_mnozica_bert/testna_mnozica_halozan/unzipped_testiraj_bert/crosloengual-bert-pytorch/', do_lower_case=True)

output_dir = 'drive/My Drive/ucenje_BERT/testna_mnozica_bert/testna_mnozica_halozan/unzipped_testiraj_bert/bert_model_naucen/'

model = BertForMaskedLM.from_pretrained(output_dir)

model.eval()


predictedStavki = ""
postavljeniStavki = ""
dontOverkillStevec = 0

stavek_za_med_postavljene = ""

dolzina = len(stavki.split("\n")) - 1

for stavek in stavki.split("\n"):
    if dontOverkillStevec > dolzina:
        break
    dontOverkillStevec += 1

    #len(stavki.split("\n"))
    if dontOverkillStevec % 20 == 0:
        print("progress:", dontOverkillStevec / dolzina * 100)

    if (len(stavek) > 1):
        stavek_za_med_postavljene = stavek

    stavek = re.sub(',', '', stavek)

    originalen_stavek = stavek.split(" ")

    stavek = stavek.lower()
    stavek = re.sub('[!@#$:;\'\"“”•«»%&ø×¹]', '', stavek)
    stavek = re.sub('[öó]', 'o', stavek)
    stavek = re.sub('[áä]', 'a', stavek)
    stavek = re.sub('[0123456789]', '1', stavek)
    stavek = re.sub('ë', 'e', stavek)
    stavek = re.sub('í', 'i', stavek)
    stavek = re.sub('ü', 'u', stavek)
    stavek = re.sub('ć', 'c', stavek)
    stavek = re.sub('ž', 'z', stavek)
    stavek = re.sub('č', 'c', stavek)
    stavek = re.sub('š', 's', stavek)
    stavek = re.sub('é', 'e', stavek)
    stavek = re.sub('đ', 'd', stavek)

    stavek = "[CLS] " + stavek + " [SEP]"

    #print(stavek)

    maskedStavek = ""
    mimozacetka = False
    for token in stavek.split(" "):
        if token != "[CLS]" and token != "[SEP]" and token != "." and token != "?" and token != "!" and mimozacetka:
            maskedStavek += " [MASK] "
        elif token != "[CLS]" and token != "[SEP]":
            mimozacetka = True
        if token == "[SEP]" or token == ".":
            maskedStavek += " "
        maskedStavek += token
        if token == "[CLS]":
            maskedStavek += " "


    #print(maskedStavek)
    input_ids = []
    attention_masks = []
    label_list = []
    # For every sentence...
    encoded_dict = tokenizer.encode_plus(
                        maskedStavek,
                        add_special_tokens = False,
                        max_length = 128,
                        pad_to_max_length = True,
                        truncation = True,
                        return_attention_mask = True,
                        return_tensors = 'pt',
                    )


    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])


    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    predictions , true_labels = [], []

    stevec = 0

    num_right_predictions = 0
    num_all_predictions = 0
    num_right_comma_predictions = 0
    num_all_commas = 0

    outputs = model(input_ids, attention_mask=attention_masks)

    logits = outputs[0]

    input_ids = input_ids[0].numpy()

    predicted_stavek = ""
    index_maske = 0

    origin_counter_stavek = 0
    index_maske_cez_mejo = False

    if len(maskedStavek.split(" ")) > 127:
        continue

    for token in maskedStavek.split(" "):
        #print(torch.argmax(logits[0, index_maske]).item())
        if index_maske > 126:
            index_maske_cez_mejo = True
            break
        if token != "[CLS]" and token != "[SEP]":
            if token == "[MASK]":
                while input_ids[index_maske] != 105:
                    index_maske = index_maske + 1
                    if index_maske > 126:
                        index_maske_cez_mejo = True
                        break
                if torch.argmax(logits[0, index_maske]).item() == 1:
                    predicted_stavek += ","
                index_maske += 1
            else:
                if index_maske > 0 and token != ".":
                    predicted_stavek += " "
                predicted_stavek += originalen_stavek[origin_counter_stavek]
                origin_counter_stavek += 1

    if index_maske_cez_mejo:
        continue

    if len(predicted_stavek) > 1 and len(stavek_za_med_postavljene) > 1:
        predictedStavki += predicted_stavek + "\n"
        postavljeniStavki += stavek_za_med_postavljene + "\n"
    stavek_za_med_postavljene = ""

    assert len(postavljeniStavki.split('\n')) == len(predictedStavki.split('\n'))

#print(postavljeniStavki)
#print("\n")
#print(predictedStavki)

if predictedStavki == "":
    predictedStavki = "Prosimo, vnesite daljše besedilo."

i = 0
splitPostavljeniStavki = postavljeniStavki.split('\n')
splitPredictedStavki = predictedStavki.split('\n')

stPravilnoPostavljenih = 0
stOdvecnoPostavljenih = 0
stPozabljenih = 0
stVseh = 0


for i in range(len(splitPostavljeniStavki)):
    #print(splitPostavljeniStavki[i])
    #print(splitPredictedStavki[i])
    originalniStavekVejice = [n for n,x in enumerate(splitPostavljeniStavki[i]) if x==',']
    pridictedStavekVejice = [n for n,x in enumerate(splitPredictedStavki[i]) if x==',']

    indexVejiceOriginalni = 0
    indexVejicePredicted = 0
    pristevalnik = 0

    stVseh += len(originalniStavekVejice)
    while True:
        if indexVejiceOriginalni >= len(originalniStavekVejice):
            if indexVejicePredicted < len(pridictedStavekVejice):
                stOdvecnoPostavljenih += len(pridictedStavekVejice) - indexVejicePredicted
            break
        if indexVejicePredicted >= len(pridictedStavekVejice):
            stPozabljenih += len(originalniStavekVejice) - indexVejiceOriginalni
            break
        if originalniStavekVejice[indexVejiceOriginalni] + pristevalnik <= pridictedStavekVejice[indexVejicePredicted]:
            if originalniStavekVejice[indexVejiceOriginalni] + pristevalnik in pridictedStavekVejice:
                stPravilnoPostavljenih += 1
                indexVejiceOriginalni += 1
                indexVejicePredicted += 1
            else:
                pristevalnik -= 1
                indexVejiceOriginalni += 1
                stPozabljenih += 1
        else:
            stOdvecnoPostavljenih += 1
            indexVejicePredicted += 1
            pristevalnik += 1




print("st vseh je:", stVseh)
print("st pravilno postavljenih je:", stPravilnoPostavljenih)
print("st odvecno postavljenih je:", stOdvecnoPostavljenih)
print("st pozabljenih je:", stPozabljenih)

KeyboardInterrupt: ignored